In [ ]:
#Importing required libraries
from __future__ import print_function
import tweepy
import json
import MySQLdb 
from dateutil import parser
import os
 
#Setting the search words, the code will grab tweets that contain below hashtags
WORDS = ['#bitcoin','#XRP','ripple','#ethereum','#BitcoinCash','#EOS','#tether','#litecoin','#stellar','#TRON','#bitcoin SV','Cardano','binance','#Monero', '#DASH']


#Defining twitter API keys and tokens 
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')
 
#Defining database details hostname, user, password and database
HOST = "data.cep8edprhf64.us-east-1.rds.amazonaws.com"
USER = os.environ.get('USER')
PASSWD = os.environ.get('PASSWD')
DATABASE = "data"
 
#Below function will select 'created_at', 'text', 'screen_name','tweet_id', 'followers' and 'friends' 
#Store it in MySQL database
def store_data(created_at, text, name, screen_name, tweet_id, followers, friends):
    db=MySQLdb.connect(host=HOST, user=USER, passwd=PASSWD, db=DATABASE, charset="utf8")
    cursor = db.cursor()
    insert_query = "INSERT INTO tweets (tweet_id, name, screen_name, created_at, text, followers, friends) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(insert_query, (tweet_id, name, screen_name, created_at, text, followers, friends))
    db.commit()
    cursor.close()
    db.close()
    return
 
#Class provided by the tweepy library to access Twitter Streaming API
class StreamListener(tweepy.StreamListener):    
    
     def on_connect(self):
        # print message once connected to the API
        print("You are now connected to the streaming API.")
 
     def on_error(self, status_code):
        #In case an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
 
     def on_data(self, data):
        #Connects to the database and stores the tweets
        try:
           # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #Extract data from the Tweet
            text = datajson['text']
            name = datajson['user']['name']
            screen_name = datajson['user']['screen_name']
            tweet_id = datajson['id']
            created_at = parser.parse(datajson['created_at']) 
            followers = datajson['user']['followers_count']
            friends = datajson['user']['friends_count']
 
            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            
            #insert the data into the MySQL database
            store_data(created_at, text, name, screen_name, tweet_id, followers, friends)
        
        except Exception as e:
           print(e)
 
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS,languages=["en"]) #filter English tweets based on search words